In [ ]:
import os
import sys
import time as t_util
import numpy as np
import cftime
import xarray as xr
import matplotlib.pyplot as plt
import scipy.stats
import matplotlib
import yaml


In [ ]:
#Read main paths
with open('../path_main.txt', 'r') as file:     path_main  = file.read()
with open('../path_grids.txt', 'r') as file:    path_grids = file.read()

dir_HWMId  = f'{path_main}Data/EURO-CORDEX/HWMId_Europe/'
dir_EMT    = f'{path_main}Data/EURO-CORDEX/EMT/'
dir_names  = f'{path_main}Scripts/Model_lists/'
dir_regrid = f'{path_grids}Regridding/'
dir_out    = f'{path_main}Data/Plot_preparation/HWMId/EURO-CORDEX_Europe/'
if not os.path.exists(dir_out): os.mkdir(dir_out)


## Prepare variables and parameters

In [ ]:
#Define models and RCPs which should be used
all_models = []
with open(dir_names + 'Models_CORDEX-EUR-11_RCP85.txt', 'r') as filehandle:
    for line in filehandle:
        all_models.append(eval(line[:-1]))

#Define scenario
RCP = 'rcp85'        
        
#Read standard grid
fname_grid_stand = dir_regrid + 'Standard_grid_CORDEX-EUR-11_NCC-NorESM1-M_SMHI-RCA4.nc'
grid_standard = xr.open_dataset(fname_grid_stand)


## Prepare variables and parameters

In [ ]:
#Define variable
HSI = 'TX'

#Define RCP
RCP = 'rcp85'

#Define EMT for regridded data
EMT_sel = '3.0K'

#Define changes in EMT relative to 1981-2010
EMT_change     = np.array([1.0, 2.0, 3.0])
EMT_change_str = ['1.0K', '2.0K', '3.0K']

#Loop over models
create = 1
for i1, model in enumerate(all_models):

    print(str(i1+1) + ') ' + "_".join(model))
    
    #Read data
    fname_CORDEX = [file for file in os.listdir(dir_HWMId) if HSI in file and RCP + '_' in file and model[0] in file and model[1] in file and model[2] in file]
    if len(fname_CORDEX)!=1:  sys.exit('File is not unique')
    data_CORDEX = xr.open_dataset(dir_HWMId + fname_CORDEX[0])

    #Read European mean temperature (EMT)
    files_EMT = sorted([dir_EMT + file for file in os.listdir(dir_EMT) if "_".join(model) in file and 'EMT_' in file])
    data_EMT  = xr.concat((xr.open_dataset(file) for file in files_EMT), dim='time')

    #Calculate EMT relative to 1981-2010 and calculate 20-year means
    dataEMT_ref = data_EMT.sel(time=slice('1981', '2010')).mean('time')
    dataEMT_rel = data_EMT - dataEMT_ref
    dataEMT_20y = dataEMT_rel.rolling(time=20, center=True).mean()

    #Loop over selected EMT levels
    create2 = 1
    for dEMT, dEMT_str in zip(EMT_change, EMT_change_str):

        #Identify 20-year period in wich level is reached for first time
        ind = np.where(dataEMT_20y.tas>dEMT)[0][0]
        central_year = dataEMT_20y.isel(time=ind).time.dt.year
        start_year   = int(central_year - 20 / 2)
        end_year     = int(central_year + (20 / 2 - 1))
        years_sel    = slice(str(start_year), str(end_year))

        if end_year>2099:
            print(model)
            print(end_year)

        #Calculate median
        data_Q50 = data_CORDEX.HWMID.sel(time=years_sel).median('time')
        data_Q50 = data_Q50.to_dataset(name='HWMID')

        #Collect in one array
        if create2==1:
            data_coll_EMT = data_Q50
            create2 = 0
        else:
            data_coll_EMT = xr.concat((data_coll_EMT, data_Q50), dim='EMT')
            
    #Add EMT string to array
    data_coll_EMT['EMT'] = EMT_change_str

    #Save in NetCDF
    fname_save = dir_out + 'HWMID-' + HSI + '_' + "_".join(model) + '_' + RCP + '_EMT.nc'
    if os.path.exists(fname_save): os.remove(fname_save)
    data_coll_EMT.to_netcdf(fname_save)

    #Save in NetCDF for regridding
    fname_tmp = dir_out + 'HWMID-' + HSI + '_' + "_".join(model) + '_' + RCP + '_EMT_tmp.nc'
    if os.path.exists(fname_tmp): os.remove(fname_tmp)
    data_coll_EMT.sel(EMT=EMT_sel).to_netcdf(fname_tmp)
    
    #Regrid to standard grid
    fname_grid = dir_regrid + 'grid_xy_CORDEX-EUR-11_NCC-NorESM1-M_SMHI-RCA4'
    fname_regrid = dir_out + "_".join(model) + HSI + "_regridded_tmp.nc"
    if os.path.exists(fname_regrid): os.remove(fname_regrid)
    os.system("cdo remapbil," + fname_grid + " " + fname_tmp + " " + fname_regrid)

    #Read regridded dataset
    data_regr = xr.open_dataset(fname_regrid)
    data_regr = data_regr.reindex({'rlat': grid_standard['rlat'], 'rlon': grid_standard['rlon']}, method='nearest')
    
    #Remove unnecessary variables
    vars_remove = set(data_regr.data_vars).difference(['HWMID'])
    data_regr = data_regr.drop(vars_remove)
    if 'height' in data_regr: data_regr = data_regr.drop('height')

    #Collect in one array
    if create==1:
        data_coll_models = data_regr
        create = 0
    else:
        data_coll_models = xr.concat((data_coll_models, data_regr), dim='model')

    #Remove temporary file
    os.remove(fname_tmp)
#     os.remove(fname_regrid)
        
#Add model names
data_coll_models['model'] = ["_".join(model) for model in all_models]

#Save data in file
fname_out = dir_out + 'HWMId-' + HSI + '_all_models_' + RCP + '_EMT' + EMT_sel + '.nc'
if os.path.exists(fname_out): os.remove(fname_out)
data_coll_models.to_netcdf(fname_out)
